# Pt Adatoms Island on MoS2

## 0. Introduction

In this example, we will create a MoS2 slab with a Pt island formed by adatoms, replication the structure from the publication:

> **Saidi, W. A.**
> "Density Functional Theory Study of Nucleation and Growth of Pt Nanoparticles on MoS2(001) Surface."
> Crystal Growth & Design, 15(2), 642–652.  (2015)
> [DOI](https://doi.org/10.1021/cg5013395)

Island formation will be performed with consecutive addition of adatom defects to reproduce Figure 4. b) material:

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/defects/defect_point_adatom_island_molybdenum_disulfide_platinum/0-figure-from-manuscript.webp" alt="Pt island on MoS2" width="400"/>

## 1. Prepare the Environment
### 1.1. Set up defect parameters 

In [ ]:

# Slab parameters
MATERIAL_NAME = "MoS2.*2D"  # Name of the material (regex) to be loaded from Standata
MILLER_INDICES = (0, 0, 1)  # Miller indices of the surface
SLAB_THICKNESS = 1  # Thickness of the slab in unit cells
VACUUM = 10.0  # Vacuum thickness in Angstrom
XY_SUPERCELL_MATRIX = [[3, 0], [0, 3]]

# Defect parameters
DEFECT_TYPE = "adatom"
PLACEMENT_METHOD = "exact_coordinate"  # Choose between "equidistant", "new_crystal_site",  "exact_coordinate"
CHEMICAL_ELEMENT = "Pt"  # Element to be placed at the site 
USE_CARTESIAN_COORDINATES = False  # If True, the position is in Cartesian coordinates

# Values below are taken from the publication:
DISTANCE_Z_1st_Pt_layer = 1.2  # Distance between S (MoS2) and Pt in first layer, in Angstrom
DISTANCE_Z_2nd_Pt_layer = 2.8  # Distance between S (MoS2) and Pt in second layer, in Angstrom

DISTANCE_Z_VALUES = [DISTANCE_Z_1st_Pt_layer, DISTANCE_Z_1st_Pt_layer, DISTANCE_Z_1st_Pt_layer, DISTANCE_Z_2nd_Pt_layer]

# In the unit cell of MoS2 atoms have S: [2/3, 1/3, 0] and Mo: [1/3, 2/3, 0] crystal coordinates.
# Creation of 3x3 supercell scales coordinates to steps of 1/9th between points: S, Mo and hollow.
# The adatoms are placed on the top of Mo atoms and S atoms can be obtained:
COORDINATES = [
    [5 / 9, 4 / 9, 0],  # 1st atom, atop central Mo
    [2 / 9, 4 / 9, 0],  # 2nd atom, next clockwise atop Mo
    [5 / 9, 7 / 9, 0],  # 3rd atom, next clockwise atop Mo
    [4 / 9, 5 / 9, 0],  # 4th topmost atom, atop S
]


### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install(
        "mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples|create_point_defect.ipynb")

### 1.3. Get input material
Materials are loaded with `get_materials()`.

In [ ]:
from utils.visualize import visualize_materials
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

substrate = Material.create(Materials.get_by_name_first_match(MATERIAL_NAME))

# Preview materials
visualize_materials([substrate])
visualize_materials([substrate], rotation="-90x")


### 1.4. Create and preview Slab

In [ ]:
from mat3ra.made.tools.helpers import create_slab
from utils.visualize import visualize_materials as visualize

slab = create_slab(
    crystal=substrate,
    termination_top=None,  # Use default termination
    miller_indices=MILLER_INDICES,
    number_of_layers=SLAB_THICKNESS,
    vacuum=VACUUM,
    use_orthogonal_c=True,
    xy_supercell_matrix=XY_SUPERCELL_MATRIX
)
visualize([{"material": slab, "rotation": "0x"}, {"material": slab, "rotation": "-90x"}], repetitions=[1, 1, 1])


## 2. Create the Defects

#### 2.1. Create the defct dictionaries

In [ ]:
from mat3ra.made.tools.build.defective_structures.two_dimensional.adatom.types import AdatomDefectDict
DEFECT_DICTS = [
    AdatomDefectDict(**{
        "coordinate_2d": coordinate[0:2],
        "distance_z": DISTANCE_Z_VALUES[i],
        "element": CHEMICAL_ELEMENT,
        "use_cartesian_coordinates": USE_CARTESIAN_COORDINATES
        # placement method is common for all adatoms
    })
    for i, coordinate in enumerate(COORDINATES)
]

### 2.2. Create the defects

In [ ]:
from mat3ra.made.tools.helpers import create_multiple_adatom_defects

slab_with_adatom = create_multiple_adatom_defects(
    slab=slab,
    defect_dicts=DEFECT_DICTS,
    placement_method=PLACEMENT_METHOD,
)


## 3. Visualize the Slabs with Adatom

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_adatom, "title": f"Material with adatom defect at {PLACEMENT_METHOD} position"}],
          viewer="wave")

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials, download_content_to_file

set_materials([slab_with_adatom])
download_content_to_file(slab_with_adatom.to_json(), "MoS2_Pt_island.json")